<a href="https://colab.research.google.com/github/jetsonai/HK_LSTMSenfuClass/blob/main/Day1/CNN/%5B3%5DPretrained_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **사전 학습된 모델을 활용한 추론 (Inference with Pretrained Network)**

## 1. 라이브러리 불러오기

In [ ]:
import os

from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from torchsummary import summary
from tqdm import tqdm

## 2. 시험 데이터셋 다운로드

In [ ]:
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
!unzip -qq cats_and_dogs_filtered.zip

### A. 데이터셋 시각화

In [ ]:
sample_cat = Image.open("/content/cats_and_dogs_filtered/validation/cats/cat.2000.jpg")
sample_dog = Image.open("/content/cats_and_dogs_filtered/validation/dogs/dog.2000.jpg")

#### i. 고양이

In [ ]:
sample_cat

#### ii. 강아지

In [ ]:
sample_dog

## 3. Custom Dataloader 생성

In [ ]:
########## Custom DataLoader ##########
class PyTorchCustomDataset(Dataset):
  def __init__(self, root_dir="cats_and_dogs_filtered/train", transform=None):
    self.image_abs_path = root_dir
    self.transform = transform
    self.label_list = os.listdir(self.image_abs_path)
    self.label_list.sort()
    self.x_list = []
    self.y_list = []
    for label_index, label_str in enumerate(self.label_list):
      img_path = os.path.join(self.image_abs_path, label_str)
      img_list = os.listdir(img_path)
      for img in img_list:
        self.x_list.append(os.path.join(img_path, img))
        self.y_list.append(label_index)
    pass

  def __len__(self):
    return len(self.x_list)

  def __getitem__(self, idx):
    image = Image.open(self.x_list[idx])
    if image.mode != "RGB":
      image = image.convert('RGB')
    if self.transform is not None:
      image = self.transform(image)
    return image, torch.tensor(self.y_list[idx]).type(torch.LongTensor)

  def __save_label_map__(self, dst_text_path="label_map.txt"):
    label_list = self.label_list
    f = open(dst_text_path, 'w')
    for i in range(len(label_list)):
      f.write(label_list[i]+'\n')
    f.close()
    pass

  def __num_classes__(self):
    return len(self.label_list)

## 4. PyTorch Pretrained Models

### A. ImageNet에 훈련된 모델 리스트
*   https://docs.pytorch.org/vision/0.21/models.html




### B. 사전 학습된 모델 불러오기

In [ ]:
from torchvision.models import alexnet, vgg16_bn
from torchvision.models import AlexNet_Weights, VGG16_BN_Weights

In [ ]:
# Check Device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Current Device : {device}")

Current Device : cuda


### C. 가중치 불러오기

In [ ]:
model_alexnet = alexnet(AlexNet_Weights.DEFAULT)
model_vgg16_bn = vgg16_bn(VGG16_BN_Weights.DEFAULT)

In [ ]:
model_alexnet = model_alexnet.eval().to(device)
model_vgg16_bn = model_vgg16_bn.eval().to(device)

### D. 모델 구조 시각화

In [ ]:
img_channels, img_size = 3, 224

#### i. AlexNet

In [ ]:
summary(model_alexnet, (img_channels, img_size, img_size))

#### ii. VGG16 with Batch-Norm.

In [ ]:
summary(model_vgg16_bn, (img_channels, img_size, img_size))

### E. Freeze Model Parameters

In [ ]:
for p in model_alexnet.parameters() :
  p.requires_grad = False

summary(model_alexnet, (img_channels, img_size, img_size))

In [ ]:
for p in model_vgg16_bn.parameters() :
  p.requires_grad = False

summary(model_vgg16_bn, (img_channels, img_size, img_size))

## 5. 사전 학습된 모델을 활용한 추론 진행

In [ ]:
########## Inference Code ##########
def inference(model, src="cats_and_dogs_filtered/validation", img_size=224, img_channels=3) :
  # Load Dataset
  transform = transforms.Compose([transforms.Resize((img_size, img_size)),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                        std=[0.229, 0.224, 0.225])]) # ImageNet의 RGB 통계량

  # Create Custom Dataset Instance
  test_dataset = PyTorchCustomDataset(src, transform)

  # Create DataLoader
  test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

  # Check Device
  device = "cuda" if torch.cuda.is_available() else "cpu"
  print(f"Current Device : {device}")

  # Create Model Instance
  model = model.eval().to(device)

  # Summarize Model
  summary(model, (img_channels, img_size, img_size))

  # Create List Instance
  cat_label_dict, dog_label_dict = {}, {}

  # Inference
  with torch.no_grad() :
    # Create TQDM Bar Instance
    test_bar = tqdm(test_loader)

    for data in test_bar :
      img, label = data
      img = img.to(device)
      pred = model(img)
      pred = torch.argmax(pred, dim=-1).cpu().item()

      if label == 0 : # 0 for Cat / 1 for Dog
        if pred not in cat_label_dict.keys() :
            cat_label_dict[pred] = 1
        else :
            cat_label_dict[pred] += 1
      else :
        if pred not in dog_label_dict.keys() :
            dog_label_dict[pred] = 1
        else :
            dog_label_dict[pred] += 1

      # Show Inference Status
      test_bar.set_description(desc=f"[Inference]")

  # Sort by Keys
  cat_label_dict_sorted = sorted(cat_label_dict.items(), key=lambda x:x[1], reverse=True)
  dog_label_dict_sorted = sorted(dog_label_dict.items(), key=lambda x:x[1], reverse=True)
  print()
  print(f"Cat Predictions : {cat_label_dict_sorted}")
  print(f"Dog Predictions : {dog_label_dict_sorted}")

### A. ImageNet Class Index (Number) List
*   https://deeplearning.cms.waikato.ac.nz/user-guide/class-maps/IMAGENET/

### B. AlexNet

In [ ]:
inference(model_alexnet)

### C. VGG16 with Batch-Norm.

In [ ]:
inference(model_vgg16_bn)

## 6. 실습

### 사전 학습된 ResNet50를 불러오고 추론을 진행해보세요.

#### A. Torchvision에서 모델 불러오기

#### B. 모델 구조 시각화

#### C. Freeze Model Parameters

#### D. Inference with ResNet50